In [1]:
using Pkg
Pkg.add(["CSV","DataFrames","CausalForest","StatsPlots","CategoricalArrays","GLM"])


    Updating registry at `C:\Users\FERNANDO\.julia\registries\General.toml`
   Resolving package versions...
   Installed ScikitLearnBase ─────────── v0.5.0
   Installed ImageIO ─────────────────── v0.6.8
   Installed ImageSegmentation ───────── v1.8.1
   Installed TiledIteration ──────────── v0.3.1
   Installed OffsetArrays ────────────── v1.17.0
   Installed AxisArrays ──────────────── v0.4.8
   Installed JpegTurbo ───────────────── v0.1.6
   Installed TiffImages ──────────────── v0.10.2
   Installed GR_jll ──────────────────── v0.73.18+0
   Installed StatsFuns ───────────────── v1.5.2
   Installed FFTW ────────────────────── v1.10.0
   Installed PNGFiles ────────────────── v0.4.4
   Installed ImageMagick ─────────────── v1.4.2
   Installed LazyGrids ───────────────── v0.5.0
   Installed GraphPlot ───────────────── v0.6.2
   Installed Sixel ───────────────────── v0.1.5
   Installed OpenEXR ─────────────────── v0.3.3
   Installed RegionTrees ─────────────── v0.3.2
   Installed LearnAP

In [1]:
# =============================
# 1) Cargar librerías y datos
# =============================
using CSV
using DataFrames
using CategoricalArrays
using StatsPlots
using CausalForest
using GLM  # para model.matrix
# Ruta del archivo
ruta = raw"C:\Users\FERNANDO\Downloads\processed.cleveland.data"

# Leer sin encabezados
df = CSV.read(ruta, DataFrame; header=false)

# Asignar nombres
rename!(df, Symbol.([
    "age","sex","cp","restbp","chol","fbs","restecg","thalach",
    "exang","oldpeak","slope","ca","thal","hd"
]))

# Revisar
first(df, 5)
size(df)





(303, 14)

In [2]:

# =============================
# 2) Preprocesamiento
# =============================

# Variables categóricas
factor_vars = [:sex,:cp,:fbs,:restecg,:exang,:slope,:ca,:thal,:hd]
foreach(v -> df[!,v] = categorical(df[!,v]), factor_vars)

unique(df.hd)


5-element Vector{Int64}:
 0
 2
 1
 3
 4

In [11]:
using DataFrames
using CategoricalArrays
using StatsModels
using CausalForest

# ============================================
# 1) Asegurar variable de tratamiento e outcome
# ============================================
df.exang = categorical(df.exang)
df.T = Int.(levelcode.(df.exang) .- 1)   # Tratamiento 0/1
df.Y = Float64.(df.thalach)

# ============================================
# 2) Seleccionar las covariables manualmente
# ============================================
covariables = setdiff(names(df), [:Y, :T, :exang])

# Convertir a matriz X (sin intercepto)
X_model = Matrix(df[:, covariables])

# ============================================
# 3) Vectores Y y T
# ============================================
T_vec = Float64.(df.T)
Y_vec = Float64.(df.Y)




303-element Vector{Float64}:
 150.0
 108.0
 129.0
 187.0
 172.0
 178.0
 160.0
 163.0
 147.0
 155.0
   ⋮
 136.0
 182.0
  90.0
 123.0
 132.0
 141.0
 115.0
 174.0
 173.0

In [23]:
using DataFrames
using CategoricalArrays
using StatsModels
using CausalForest

# ============================================
# 1) Variables de tratamiento y outcome
# ============================================
df.exang = categorical(df.exang)
df.T = Int.(levelcode.(df.exang) .- 1)     # Tratamiento
df.Y = Float64.(df.thalach)                # Outcome

# ============================================
# 2) Covariables
# ============================================
covariables = setdiff(names(df), [:Y, :T, :exang])
df2 = df[:, covariables]

# Convertir categóricas y asegurar Float64
for c in names(df2)
    if df2[!, c] isa CategoricalVector
        df2[!, c] = Float64.(levelcode.(df2[!, c]))
    else
        df2[!, c] = Float64.(df2[!, c])
    end
end

X = Matrix{Float64}(df2)
T = Int64.(df.T)
Y = Float64.(df.Y)

println(size(X)) # confirm dimensions

# ============================================
# 3) Entrenar Causal Forest
# ============================================
n_trees = 2000
max_depth = 10

cf = build_forest(
    false,       # bootstrap
    true,        # honest splitting
    Y,
    T,
    X,
    true,        # const_mtry
    2,           # m_pois (antes era nothing ⚠️)
    n_trees,     # número de árboles
    10           # n_trees_centering (antes nothing ⚠️)
)

# ============================================
# 4) Predicción efecto causal
# ============================================
τ_hat = apply_forest(cf, X)
ATE = mean(τ_hat)

println("\n✅ Bosque causal entrenado correctamente")
println("🎯 ATE estimado = $ATE")


(303, 16)

✅ Bosque causal entrenado correctamente
🎯 ATE estimado = -2.5311435488453786


In [26]:
using Plots
using DataFrames

# =============================
# 5) Visualización CATE
# =============================
histogram(
    τ_hat,                             # <- tu vector de efectos
    bins = 40,
    title = "Distribución de CATE",
    xlabel = "CATE estimado",
    ylabel = "Frecuencia",
    legend = false
)

# =============================
# Importancia de variables
# =============================
var_imp = importance(cf)               # función correcta en CausalForest.jl

var_imp_df = DataFrame(
    variable = covariables,            # <- las mismas covariables usadas en X
    importance = var_imp
)

first(sort(var_imp_df, :importance, rev=true), 10)


Row,variable,importance
,Any,Float64
1,age,0.124362
2,chol,0.105523
3,restbp,0.0959011
4,oldpeak,0.0905727
5,Y,0.0846162
6,thalach,0.082121
7,hd,0.0661189
8,sex,0.0630999
9,thal,0.0575167


In [32]:
using CategoricalArrays

for col in covariables
    # Si es texto, primero convertir a Categorical
    if eltype(df[!, col]) <: AbstractString
        df[!, col] = categorical(df[!, col])
    end
    
    # Si es categoría, codificar a Int
    if df[!, col] isa CategoricalArray
        df[!, col] = Int.(levelcode.(df[!, col]))
    end
end

tree_data = Matrix(df[:, covariables])
tree_labels = df.cate_hat

model_tree = DecisionTree.DecisionTreeRegressor(max_depth = 2, min_samples_leaf = 20)
DecisionTree.fit!(model_tree, tree_data, tree_labels)

println("\n🧠 Árbol de decisión representativo del CATE:\n")
DecisionTree.print_tree(model_tree, 4)




🧠 Árbol de decisión representativo del CATE:

Feature 1 < 58.0 ?
├─ Feature 2 < 0.5 ?
    ├─ -2.573771589263763 : 0/51
    └─ -2.952524087182879 : 0/128
└─ Feature 2 < 0.5 ?
    ├─ -1.8800654221301845 : 0/46
    └─ -2.1957442521833426 : 0/78


In [33]:
# =============================
# 7) Subgrupos ejemplo
# =============================
combine(groupby(df, :sex), :cate_hat => mean => :mean_cate, nrow => :n)

Row,sex,mean_cate,n
,Int64,Float64,Int64
1,0,-2.2448,97
2,1,-2.66598,206
